<a href="https://colab.research.google.com/github/gowtamvamsi/Disaster-tweet-classification/blob/master/Disaster_tweet_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
data_train = pd.read_csv("/content/train.csv")
data_test = pd.read_csv("/content/test.csv")

In [2]:
print(data_train.shape)

(7613, 5)


In [3]:
print(data_train['location'].value_counts())

USA                       104
New York                   71
United States              50
London                     45
Canada                     29
                         ... 
Montana, USA                1
In your head                1
Milton Keynes, England      1
KLA,Uganda                  1
Overland Park, KS           1
Name: location, Length: 3341, dtype: int64


In [4]:
print(data_train.head())
print(data_test.head())

   id keyword  ...                                               text target
0   1     NaN  ...  Our Deeds are the Reason of this #earthquake M...      1
1   4     NaN  ...             Forest fire near La Ronge Sask. Canada      1
2   5     NaN  ...  All residents asked to 'shelter in place' are ...      1
3   6     NaN  ...  13,000 people receive #wildfires evacuation or...      1
4   7     NaN  ...  Just got sent this photo from Ruby #Alaska as ...      1

[5 rows x 5 columns]
   id keyword location                                               text
0   0     NaN      NaN                 Just happened a terrible car crash
1   2     NaN      NaN  Heard about #earthquake is different cities, s...
2   3     NaN      NaN  there is a forest fire at spot pond, geese are...
3   9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
4  11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
data_train['location'] = data_train.location.apply(lambda x: x if not pd.isnull(x) else 'World')
data_test['location'] = data_test.location.apply(lambda x: x if not pd.isnull(x) else 'World')

In [6]:
print(data_train.isna().any())

id          False
keyword      True
location    False
text        False
target      False
dtype: bool


In [7]:
import os
import re
from tqdm import tqdm_notebook as tqdm
import nltk
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
def remove_speical_characters(text):
    text = re.sub(r"[^a-zA-Z !]+", '', text)
    return text

In [8]:
# Deconracting
data_train['text'] = data_train.text.apply(lambda x: decontracted(x))
data_test['text'] = data_test.text.apply(lambda x: decontracted(x))

# Removing numbers and special characters except spcae and exclamation.
#data_train['text'] = data_train.text.apply(lambda x: remove_speical_characters(x))
#data_test['text'] = data_test.text.apply(lambda x: remove_speical_characters(x))

In [9]:
print(data_train.head())

   id keyword  ...                                               text target
0   1     NaN  ...  Our Deeds are the Reason of this #earthquake M...      1
1   4     NaN  ...             Forest fire near La Ronge Sask. Canada      1
2   5     NaN  ...  All residents asked to  ishelter in place' are...      1
3   6     NaN  ...  13,000 people receive #wildfires evacuation or...      1
4   7     NaN  ...  Just got sent this photo from Ruby #Alaska as ...      1

[5 rows x 5 columns]


In [10]:
data_train['keyword'] = data_train.keyword.apply(lambda x: x if not pd.isnull(x) else 'nothing')
data_test['keyword'] = data_test.keyword.apply(lambda x: x if not pd.isnull(x) else 'nothing')

In [11]:
from sklearn.model_selection import train_test_split
y_train = data_train['target'].values
data_train.drop(['target'], axis=1, inplace=True)
print(data_train.columns)
X_train, X_cv, y_train, y_cv = train_test_split(data_train, y_train, test_size=0.2, random_state=42)

Index(['id', 'keyword', 'location', 'text'], dtype='object')


In [12]:
print(X_train.columns)
print(X_cv.shape)
print(y_train.shape)
print(y_cv.shape)

Index(['id', 'keyword', 'location', 'text'], dtype='object')
(1523, 4)
(6090,)
(1523,)


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import OneHotEncoder
vectorizer = TfidfVectorizer(stop_words={'english'}, min_df = 0.0001)
vectorizer_cnt = CountVectorizer(binary=True, ngram_range=(1,2))
vectorizer_cnt.fit(X_train['text'].values)
data_text_train = vectorizer_cnt.transform(X_train['text'].values)
data_text_cv = vectorizer_cnt.transform(X_cv['text'].values)
data_text_test = vectorizer_cnt.transform(data_test['text'].values)

enc_location = OneHotEncoder(handle_unknown='ignore')
enc_location.fit(X_train['location'].values.reshape(-1, 1))
data_location_train = enc_location.transform(X_train['location'].values.reshape(-1, 1))
data_location_cv = enc_location.transform(X_cv['location'].values.reshape(-1, 1))
data_location_test = enc_location.transform(data_test['location'].values.reshape(-1, 1))

enc_keyword = OneHotEncoder(handle_unknown='ignore')
enc_keyword.fit(X_train['keyword'].values.reshape(-1, 1))
data_keyword_train = enc_location.transform(X_train['keyword'].values.reshape(-1, 1))
data_keyword_cv = enc_location.transform(X_cv['keyword'].values.reshape(-1, 1))
data_keyword_test = enc_location.transform(data_test['keyword'].values.reshape(-1, 1))

In [14]:
print(data_text_train.shape)
print(data_text_cv.shape)
print(data_text_test.shape)

(6090, 75467)
(1523, 75467)
(3263, 75467)


In [15]:
print(data_location_train.shape)
print(data_location_cv.shape)
print(data_location_test.shape)

(6090, 2771)
(1523, 2771)
(3263, 2771)


In [16]:
print(data_keyword_train.shape)
print(data_keyword_cv.shape)
print(data_keyword_test.shape)

(6090, 2771)
(1523, 2771)
(3263, 2771)


In [17]:
from scipy.sparse import hstack
final_data_train = hstack([data_text_train,data_keyword_train])
final_data_cv = hstack([data_text_cv,data_keyword_cv])
final_data_test = hstack([data_text_test,data_keyword_test])

In [18]:
print(final_data_train.shape)
print(final_data_cv.shape)
print(final_data_test.shape)

(6090, 78238)
(1523, 78238)
(3263, 78238)


In [19]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [20]:
from sklearn.linear_model import LogisticRegression #importing logistic regression model

lr = LogisticRegression()

lr.fit(final_data_train,y_train)#fit or train data

y_cv_pred = lr.predict(final_data_cv)
y_cv_pred_prob_lr = lr.predict_proba(final_data_cv)
print('Logistic Regression cv accuracy : ',accuracy_score(y_cv_pred,y_cv))
print('Logistic Regression cv AUC : ',roc_auc_score(y_cv_pred,y_cv))

Logistic Regression cv accuracy :  0.8128693368351937
Logistic Regression cv AUC :  0.8165028582516156


In [21]:
print(y_cv_pred_prob_lr)

[[0.99153806 0.00846194]
 [0.77362302 0.22637698]
 [0.4024781  0.5975219 ]
 ...
 [0.06948519 0.93051481]
 [0.01020063 0.98979937]
 [0.20957346 0.79042654]]


In [22]:
y_test_pred = lr.predict(final_data_test)
print(y_test_pred)
data_test['target'] = y_test_pred
print(data_test.head())

[1 1 1 ... 1 1 0]
   id  keyword  ...                                               text target
0   0  nothing  ...                 Just happened a terrible car crash      1
1   2  nothing  ...  Heard about #earthquake is different cities, s...      1
2   3  nothing  ...  there is a forest fire at spot pond, geese are...      1
3   9  nothing  ...           Apocalypse lighting. #Spokane #wildfires      0
4  11  nothing  ...      Typhoon Soudelor kills 28 in China and Taiwan      1

[5 rows x 5 columns]


In [23]:
data_test.drop(['keyword','location', 'text'], axis=1, inplace=True)
print(data_test.head())
data_test.to_csv('submission_file_1.csv', index=False)

   id  target
0   0       1
1   2       1
2   3       1
3   9       0
4  11       1


In [24]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(final_data_train,y_train)

y_cv_pred = knn.predict(final_data_cv)
print('KNN cv accuracy : ',accuracy_score(y_cv_pred,y_cv))
print('KNN cv AUC : ',roc_auc_score(y_cv_pred,y_cv))

KNN cv accuracy :  0.6730137885751806
KNN cv AUC :  0.7878377221151787


In [25]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf', C=1000, gamma ='auto', probability = True)

svm.fit(final_data_train,y_train)

y_cv_pred = svm.predict(final_data_cv)
y_cv_pred_prob = lr.predict_proba(final_data_cv)
print('SVM cv accuracy : ',accuracy_score(y_cv_pred,y_cv))
print('SVM cv AUC : ',roc_auc_score(y_cv_pred,y_cv))

SVM cv accuracy :  0.8161523309258043
SVM cv AUC :  0.8332697693820392


In [26]:
print(y_cv_pred_prob)

[[0.99153806 0.00846194]
 [0.77362302 0.22637698]
 [0.4024781  0.5975219 ]
 ...
 [0.06948519 0.93051481]
 [0.01020063 0.98979937]
 [0.20957346 0.79042654]]


In [27]:
y_cv_mod_pred = []
for i in range(len(y_cv_pred)):
    max_val = -1
    cl = 0
    if(y_cv_pred_prob[i][0]>max_val):
        max_val = y_cv_pred_prob[i][0]
        cl = 0
    if(y_cv_pred_prob[i][1]>max_val):
        max_val = y_cv_pred_prob[i][1]
        cl = 1
    if(y_cv_pred_prob_lr[i][0]>max_val):
        max_val = y_cv_pred_prob_lr[i][0]
        cl = 0
    if(y_cv_pred_prob_lr[i][1]>max_val):
        max_val = y_cv_pred_prob_lr[i][1]
        cl = 1
    y_cv_mod_pred.append(cl)

In [28]:
print('SVM cv accuracy : ',accuracy_score(y_cv_mod_pred,y_cv))
print('SVM cv AUC : ',roc_auc_score(y_cv_mod_pred,y_cv))

SVM cv accuracy :  0.8128693368351937
SVM cv AUC :  0.8165028582516156


In [29]:
y_test_pred = svm.predict(final_data_test)
data_test['target'] = y_test_pred
print(data_test.head())
data_test.to_csv('submission_file_svc_2.csv', index=False)

   id  target
0   0       0
1   2       0
2   3       1
3   9       0
4  11       1


In [30]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(final_data_train.toarray(),y_train)

y_cv_pred = nb.predict(final_data_cv.toarray())
print('Naive bayes cv accuracy : ',accuracy_score(y_cv_pred,y_cv))
print('Naive bayes cv AUC : ',roc_auc_score(y_cv_pred,y_cv))

Naive bayes cv accuracy :  0.7071569271175312
Naive bayes cv AUC :  0.7089648294615374


In [31]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dt.fit(final_data_train,y_train)

y_cv_pred = dt.predict(final_data_cv)
print('Decision Tree cv accuracy : ',accuracy_score(y_cv_pred,y_cv))
print('Decision Tree cv AUC : ',roc_auc_score(y_cv_pred,y_cv))

Decision Tree cv accuracy :  0.7491792514773473
Decision Tree cv AUC :  0.7455815842627558


In [32]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 100,random_state = 40)

rf.fit(final_data_train,y_train)

y_cv_pred = rf.predict(final_data_cv)
print('Random Forest cv accuracy : ',accuracy_score(y_cv_pred,y_cv))
print('Random Forest cv AUC : ',roc_auc_score(y_cv_pred,y_cv))

Random Forest cv accuracy :  0.7793827971109653
Random Forest cv AUC :  0.8157772099287363
